In [1]:
from mpl_toolkits.mplot3d import Axes3D

In [2]:
# import libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
import numpy as np
import pandas as pd
import random
import sklearn
import time
import math
import matplotlib.pyplot as plt
#import os
import sys

from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import decomposition
from sklearn import manifold
from tqdm import tqdm
from scipy import stats
#from torchviz import make_dot
from utils import *

In [3]:
robot_choice = "7DoF-7R-Panda"
network_type = "MLP"                                                    # MLP, ResMLP, DenseMLP, FouierMLP 
optimizer_choice = "SGD"                                                      # optimizers (SGD, Adam, Adadelta, RMSprop)
loss_choice = "l2"                                                            # l2, l1, lfk
experiment_number = 1
scale = 2
dataset_samples = 10000
device = torch.device('cuda:'+str("0")) 


seed = True                                                                   # seed random generators for reproducibility
visualize_joints = True                                                       # visualize joint distribution in dataset     
visualize_workspace = True                                                    # visualize workspace (positions)
visualize_losses = True                                                       # visuallze training and validation losses
visualize_normalized_workspace = True                                         # visualize normalized workspace (positions - debugging purposes)
visualize_workspace_results = True                                            # visualize results in workapce
print_inference_summary = True                                                # perform and print inference summary after training is done
print_epoch = True  
batch_size = 128                                                              # desired batch size
init_type = "default"                                                         # weights init method (default, uniform, normal, xavier_uniform, xavier_normal)
#hidden_layer_sizes = [128,128,128,128]                                           # architecture to employ
learning_rate = 1e-4                                                          # learning rate

EPOCHS = 10000                                                                # total training epochs


if robot_choice == "7DoF-7R-Panda":
    n_DoF = 7
    input_dim = 6
    output_dim = 7
    pose_header = ["x", "y", "z","R","P","Y"]
    joint_header = ["t1", "t2", "t3", "t4", "t5", "t6", "t7"]

In [5]:
# loop through layers and neurons

#layers = 1
#neuron = 100

avg_pos_arr = np.zeros((5,10,5))
avg_ori_arr = np.zeros((5,10,5))




for scale in range(2,12,2):

    print("\nSummary of the results being evaluated: ")
    print("==> robot choice: {}".format(robot_choice))
    print("==> scale: {}".format(scale))
    
    for layers in range(1,6,1):
        
        for neuron in range(100,1100,100):
            
            hidden_layer_sizes = np.zeros((1,layers)) 
            hidden_layer_sizes[:,:] = neuron
            hidden_layer_sizes = hidden_layer_sizes.squeeze(0).astype(int).tolist()
            
            
            architecture = "{}-{}-{}".format(str(input_dim),
                                             str(hidden_layer_sizes).replace(" ","").replace("[","").replace("]","").replace(",","-"),
                                             str(output_dim)
                                            )
            print("==> architecture: {}".format(architecture))


            save_path = "../../ikresults/"+robot_choice+"/"+robot_choice+"_" \
            +network_type+"_"+architecture+"_" \
            +optimizer_choice+"_"+loss_choice+"_"+str(experiment_number)+'_qlim_scale_'+str(int(scale))+'_samples_'+str(dataset_samples)

            X_test = np.array(pd.read_csv(save_path+"/X_test_"+robot_choice+"_"+str(dataset_samples)+".csv"))
            y_test = np.array(pd.read_csv(save_path+"/y_test_"+robot_choice+"_"+str(dataset_samples)+".csv"))
            
            test_data_loader = load_test_dataset(X_test, y_test)
            


            
            # set loss
            if loss_choice == "l2":
                criterion = nn.MSELoss(reduction="mean")
            elif loss_choice == "l1":
                criterion = nn.L1Loss(reduction="mean")
            elif loss_choice == "lfk":
                criterion = FKLoss()
                                    
            # inference
            if print_inference_summary:
                weights_file = save_path+"/best_epoch.pth"
                if network_type == "MLP":
                    model = MLP(input_dim, hidden_layer_sizes, output_dim).to(device)
                    #model = MLP(mapping_size*2, hidden_layer_sizes, output_dim).to(device)
                elif network_type == "ResMLP":
                    model = ResMLP(input_dim, hidden_layer_sizes, output_dim).to(device)
                elif network_type == "DenseMLP":
                    model = DenseMLP(input_dim, hidden_layer_sizes, output_dim).to(device)
                
                state_dict = model.state_dict()
                for n, p in torch.load(weights_file, map_location=lambda storage, loc: storage).items():
                    if n in state_dict.keys():
                        state_dict[n].copy_(p)
                    else:
                        raise KeyError(n)
                
                # get the results from training
                
                results = inference(model, test_data_loader, criterion, device, robot_choice)
                y_preds = results["y_preds"]
                X_preds = results["X_preds"]
                y_desireds = results["y_desireds"]
                X_desireds = results["X_desireds"]
                X_errors = results["X_errors"]
                
                #print("==> y_preds {}: min = {:.4f} / mean = {:.4f} / max = {:.4f}\n{}\n".format(y_preds.shape, y_preds.min(), y_preds.mean(), y_preds.max(), y_preds))
                #print("==> y_test {}: min = {:.4f} / mean = {:.4f} / max = {:.4f}\n{}\n".format(y_test.shape, y_test.min(), y_test.mean(), y_test.max(), y_test))
                
                #print("==> X_preds {}: min = {:.4f} / mean = {:.4f} / max = {:.4f}\n{}\n".format(X_preds.shape, X_preds.min(), X_preds.mean(), X_preds.max(), X_preds))
                #print("==> X_test {}: min = {:.4f} / mean = {:.4f} / max = {:.4f}\n{}\n".format(X_test.shape, X_test.min(), X_test.mean(), X_test.max(), X_test))
                
                
                with np.printoptions(precision=4, suppress=True):
                    X_errors_r = X_errors[:,:6]
                    X_errors_r[:,:3] = X_errors_r[:,:3] * 1000
                    X_errors_r[:,3:] = np.rad2deg(X_errors_r[:,3:])        
                    #print(X_errors_r[:,:6]) 
                    
                    avg_pos = X_errors_r[1,:3].mean()
                    avg_ori = X_errors_r[1,3:].mean()
                    print("avg_pos: {:.4f} (mm)".format(avg_pos))
                    print("avg_ori: {:.4f} (deg)".format(avg_ori))
                    print("\n")
                    
                    avg_pos_arr[int(scale/2)-1, int(neuron/100)-1, int(layers)-1] = avg_pos
                    avg_ori_arr[int(scale/2)-1, int(neuron/100)-1, int(layers)-1] = avg_ori


    print("Recap Position Errors: ")
    print(avg_pos_arr)
    print("\nRecap Orientation Errors: ")
    print(avg_ori_arr)
        
    #print(X_errors[:6,:].T*1000)
    #print("X_errors (mm): \n{}".format(X_errors*1000))

    #results = inference_FK(model, test_data_loader, criterion, device)
    #y_preds = results["y_preds"]
    #y_desireds = results["y_desireds"]
    #y_errors = results["y_errors"]
    
    #print("==> y_preds {}: min = {:.4f} / mean = {:.4f} / max = {:.4f}\n{}\n".format(y_preds.shape, y_preds.min(), y_preds.mean(), y_preds.max(), y_preds))
    #print("==> y_test {}: min = {:.4f} / mean = {:.4f} / max = {:.4f}\n{}\n".format(y_test.shape, y_test.min(), y_test.mean(), y_test.max(), y_test))
    #print("y_errors (mm): \n{}".format(y_errors*1000))
    
    
    #X_errors_p = np.abs(X_preds - X_desireds)*1000





Summary of the results being evaluated: 
==> robot choice: 7DoF-7R-Panda
==> scale: 2
==> architecture: 6-100-7
==> Shape X_test:  (900, 6)
==> Shape y_test:  (900, 7)
avg_pos: 14.7028 (mm)
avg_ori: 17.3667 (deg)


==> architecture: 6-200-7
==> Shape X_test:  (900, 6)
==> Shape y_test:  (900, 7)
avg_pos: 10.2421 (mm)
avg_ori: 9.9032 (deg)


==> architecture: 6-300-7
==> Shape X_test:  (900, 6)
==> Shape y_test:  (900, 7)
avg_pos: 9.1301 (mm)
avg_ori: 10.4585 (deg)


==> architecture: 6-400-7
==> Shape X_test:  (900, 6)
==> Shape y_test:  (900, 7)
avg_pos: 8.8556 (mm)
avg_ori: 9.1957 (deg)


==> architecture: 6-500-7
==> Shape X_test:  (900, 6)
==> Shape y_test:  (900, 7)
avg_pos: 8.7528 (mm)
avg_ori: 9.8643 (deg)


==> architecture: 6-600-7
==> Shape X_test:  (900, 6)
==> Shape y_test:  (900, 7)
avg_pos: 8.5400 (mm)
avg_ori: 9.2942 (deg)


==> architecture: 6-700-7
==> Shape X_test:  (900, 6)
==> Shape y_test:  (900, 7)
avg_pos: 8.1588 (mm)
avg_ori: 9.0893 (deg)


==> architecture: 6-

In [ ]:
# plots